# Introduction
Machine learning competitions are a great way to improve your data science skills and measure your progress. 

In this exercise, you will create and submit predictions for a Kaggle competition. You can then improve your model (e.g. by adding features) to improve and see how you stack up to others taking this course.

The steps in this notebook are:
1. Build a Random Forest model with all of your data (**X** and **y**)
2. Read in the "test" data, which doesn't include values for the target.  Predict home values in the test data with your Random Forest model.
3. Submit those predictions to the competition and see your score.
4. Optionally, come back to see if you can improve your model by adding features or changing your model. Then you can resubmit to see how that stacks up on the competition leaderboard.

## Recap
Here's the code you've written so far. Start by running it again.

In [3]:
def str2cols(df, column, col_vals, prefix):
    '''
    df: pandas DataFrame
    column: string (name of original column)
    col_vals: list of string (unique value in original column)
    prefix: string
    
    return: None  (modify df)
    '''
    
    for col_val in col_vals:
        df[prefix + col_val] = (df[column] == col_val).astype('int64')
        
    return

In [5]:
def add_feature(home_data):
    #home_data['Price_per_SF'] = home_data.SalePrice / \
    #                           (home_data['1stFlrSF'] + home_data['2ndFlrSF'] + home_data['TotalBsmtSF'])

    str2cols(home_data, 'SaleType', ['WD', 'New', 'COD'], 'ST_')
    
    sale_condition = ['Normal', 'Abnorml', 'Partial', 'AdjLand', 'Alloca', 'Family']
    str2cols(home_data, 'SaleCondition', sale_condition, 'SC_')

    bldg = ['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs']
    str2cols(home_data, 'BldgType', bldg, 'BT_')
    
    house_style = ['2Story', '1Story', '1.5Fin', 'SFoyer', 'SLvl']
    str2cols(home_data, 'HouseStyle', house_style, 'HS_')
        
    return

In [23]:
home_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [6]:
# Code you have previously used to load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression



# Path of the file to read. We changed the directory structure to simplify submitting to a competition
iowa_file_path = '../input/train.csv'

home_data = pd.read_csv(iowa_file_path)

# Create target object and call it y
y = home_data.SalePrice
# Create X
add_feature(home_data)
# home_data['YearBuilt'] = 2011 - home_data['YearBuilt']  # degrade in RF, no change in LR

features = ['OverallQual', 'OverallCond', 'LotArea',
            'ST_WD', 'ST_New', 'ST_COD',  'SC_Abnorml', 'SC_Partial', # 'SC_Normal',
            'MSSubClass',
            'GarageCars', # 'GarageArea',
            'YearBuilt',  # 'YearRemodAdd', 'YrSold', 
            # 'BT_1Fam', 'BT_2fmCon', 'BT_Duplex', 'BT_TwnhsE', 'BT_Twnhs',
            # 'HS_2Story', 'HS_1Story', 'HS_1.5Fin', 'HS_SFoyer', 'HS_SLvl',
            '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd']
X = home_data[features]

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

"""
# Specify Model
iowa_model = DecisionTreeRegressor(random_state=1)
# Fit Model
iowa_model.fit(train_X, train_y)

# Make validation predictions and calculate mean absolute error
val_predictions = iowa_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE when not specifying max_leaf_nodes: {:,.0f}".format(val_mae))

# Using best value for max_leaf_nodes
iowa_model = DecisionTreeRegressor(max_leaf_nodes=100, random_state=1)
iowa_model.fit(train_X, train_y)
val_predictions = iowa_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE for best value of max_leaf_nodes: {:,.0f}".format(val_mae))
"""
# Define the model. Set random_state to 1
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))


scaler = StandardScaler()
train_Xnorm = scaler.fit_transform(train_X)
val_Xnorm = scaler.transform(val_X)
"""
svm_model = SVR(kernel='linear')
svm_model.fit(train_Xnorm, train_y)
svm_val_predict = svm_model.predict(val_Xnorm)
svm_val_mae = mean_absolute_error(svm_val_predict, val_y)
print('Validation MAE for SVM: {}'.format(svm_val_mae))
"""

lr_model = LinearRegression()
lr_model.fit(train_X, train_y)
lr_val_predict = lr_model.predict(val_X)
lr_val_mae = mean_absolute_error(lr_val_predict, val_y)
print('Validation MAE for Linear Regression: {:,.0f}'.format(lr_val_mae))

print(rf_val_predictions[:5])
print(np.round(lr_val_predict[:5]))
print(val_y[:5])
#print(val_X[:5])

/home/kato/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/kato/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/kato/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/kato/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.


Validation MAE for Random Forest Model: 19,180
Validation MAE for Linear Regression: 22,006
[190483.5 142640.  119175.   78650.  131735. ]
[218428. 155531. 107175.  50661. 120946.]
258     231500
267     179500
288     122000
649      84500
1233    142000
Name: SalePrice, dtype: int64


In [19]:
home_data.SaleType.value_counts()

WD       1267
New       122
COD        43
ConLD       9
ConLw       5
ConLI       5
CWD         4
Oth         3
Con         2
Name: SaleType, dtype: int64

In [7]:
# SVR: SVM regressor: linear kernel
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

params = {"C":np.logspace(3, 3.5, 5), "epsilon":np.logspace(2, 3, 5)}
gridsearch = GridSearchCV(SVR(kernel='linear'), params, cv=4, scoring="neg_mean_absolute_error", return_train_score=True)
gridsearch.fit(train_Xnorm, train_y)
print("Best Parm =", gridsearch.best_params_, "score =", gridsearch.best_score_)
print()

Best Parm = {'C': 1778.2794100389228, 'epsilon': 562.341325190349} score = -22007.143428777814



/home/kato/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [8]:
# use best alpha and train using while training data
regr = SVR(kernel='linear', C=gridsearch.best_params_["C"], epsilon=gridsearch.best_params_['epsilon'])
regr.fit(train_Xnorm, train_y)
val_p = regr.predict(val_Xnorm)
val_mae = mean_absolute_error(val_y, val_p)
print('Validation MAE for SVR linear-kernel: {:,.0f}'.format(val_mae))

Validation MAE for SVR linear-kernel: 21,053


In [9]:
# SVR: SVM regressor: rbf(Gaussian) kernel
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

params = {"C":np.logspace(6, 7, 5), "epsilon":np.logspace(3, 4, 5)}
gridsearch = GridSearchCV(SVR(), params, cv=5, scoring="neg_mean_absolute_error", return_train_score=True, n_jobs=-1, verbose=1)
gridsearch.fit(train_Xnorm, train_y)
print("Best Parm =", gridsearch.best_params_, "score =", gridsearch.best_score_)
print()

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   43.6s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  4.7min finished


Best Parm = {'C': 1000000.0, 'epsilon': 5623.413251903491} score = -20942.35949875747



In [10]:
# use best alpha and train using while training data
regr = SVR(C=gridsearch.best_params_["C"], epsilon=gridsearch.best_params_['epsilon'])
regr.fit(train_Xnorm, train_y)
val_p = regr.predict(val_Xnorm)
val_mae = mean_absolute_error(val_y, val_p)
print('Validation MAE for SVR linear-kernel: {:,.0f}'.format(val_mae))

Validation MAE for SVR linear-kernel: 17,445


# Creating a Model For the Competition

Build a Random Forest model and train it on all of **X** and **y**.  

In [11]:
# To improve accuracy, create a new Random Forest model which you will train on all training data
rf_model_on_full_data = RandomForestRegressor(random_state=1)

# fit rf_model_on_full_data on all data from the 
rf_model_on_full_data.fit(X, y)


/home/kato/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [12]:
# Fit with SVR

Xnorm = scaler.transform(X)
regr.fit(Xnorm, y)

/home/kato/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/kato/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1000000.0, cache_size=200, coef0=0.0, degree=3,
  epsilon=5623.413251903491, gamma='auto_deprecated', kernel='rbf',
  max_iter=-1, shrinking=True, tol=0.001, verbose=False)

# Make Predictions
Read the file of "test" data. And apply your model to make predictions

In [13]:
# path to file you will use for predictions
test_data_path = '../input/test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)
test_data['GarageCars'].fillna(0.0, inplace=True)
add_feature(test_data)

# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features
test_X = test_data[features]

# make predictions which we will submit. 
test_preds = rf_model_on_full_data.predict(test_X)

# The lines below shows you how to save your data in the format needed to score it in the competition
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})

output.to_csv('submission.csv', index=False)

In [14]:
test_Xn = scaler.transform(test_X)
test_preds = regr.predict(test_Xn)
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})

output.to_csv('submission.csv', index=False)

/home/kato/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [25]:

test_data.GarageCars.fillna(0, inplace=True)
test_data[features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 17 columns):
OverallQual     1459 non-null int64
OverallCond     1459 non-null int64
LotArea         1459 non-null int64
ST_WD           1459 non-null int64
ST_New          1459 non-null int64
ST_COD          1459 non-null int64
SC_Abnorml      1459 non-null int64
SC_Partial      1459 non-null int64
MSSubClass      1459 non-null int64
GarageCars      1459 non-null float64
YearBuilt       1459 non-null int64
1stFlrSF        1459 non-null int64
2ndFlrSF        1459 non-null int64
FullBath        1459 non-null int64
BedroomAbvGr    1459 non-null int64
KitchenAbvGr    1459 non-null int64
TotRmsAbvGrd    1459 non-null int64
dtypes: float64(1), int64(16)
memory usage: 193.9 KB


# Test Your Work
After filling in the code above:
1. Click the **Commit and Run** button. 
2. After your code has finished running, click the small double brackets **<<** in the upper left of your screen.  This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
3. Go to the output tab at top of your screen. Select the button to submit your file to the competition.  
4. If you want to keep working to improve your model, select the edit button. Then you can change your model and repeat the process.

Congratulations, you've started competing in Machine Learning competitions.

# Continuing Your Progress
There are many ways to improve your model, and **experimenting is a great way to learn at this point.**

The best way to improve your model is to add features.  Look at the list of columns and think about what might affect home prices.  Some features will cause errors because of issues like missing values or non-numeric data types. 

Level 2 of this course will teach you how to handle these types of features. You will also learn to use **xgboost**, a technique giving even better accuracy than Random Forest.


# Other Courses
The **[Pandas course](https://kaggle.com/Learn/Pandas)** will give you the data manipulation skills to quickly go from conceptual idea to implementation in your data science projects. 

You are also ready for the **[Deep Learning](https://kaggle.com/Learn/Deep-Learning)** course, where you will build models with better-than-human level performance at computer vision tasks.

---
**[Course Home Page](https://www.kaggle.com/learn/machine-learning)**

**[Learn Discussion Forum](https://kaggle.com/learn-forum)**.
